In [ ]:
import pandas as pd
import numpy as np

prot_df = pd.read_csv("/Users/vitran/Documents/Work/Github/Insectipep/data/final_target_list.csv")
for _,row in prot_df.iterrows():
    seq = row["Sequence"]
    if len(seq) < 50:
        prot_df.at[_, "Group"] = "S"
    elif len(seq) < 150:
        prot_df.at[_, "Group"] = "M"
    else:
        prot_df.at[_, "Group"] = "L"


In [ ]:
import os
import shutil
import pandas as pd

# Load the dataframe
prot_df = pd.read_csv("/Users/vitran/Documents/Work/Github/Insectipep/data/final_target_list.csv")

# Classify by sequence length
def classify_len(seq):
    l = len(seq)
    if l <= 400:
        return "S"
    elif l <= 700:
        return "M"
    else:
        return "L"

prot_df["Group"] = prot_df["Sequence"].apply(classify_len)

# Set chunk sizes
chunk_sizes = {"S": 60, "M": 30, "L": 15}

# Folder containing .fa files
fa_folder = "/Users/vitran/Documents/Work/Github/ColabFold_data/insectipep"
output_base = "/Users/vitran/Documents/Work/Github/ColabFold_data/insectipep/split"

# Ensure output base exists
os.makedirs(output_base, exist_ok=True)

for group in ["S", "M", "L"]:
    # Get pdbs for this group
    pdbs = prot_df[prot_df["Group"] == group]["PDB"].tolist()
    # Find matching .fa files
    fa_files = [f for f in os.listdir(fa_folder) if f.endswith(".fa") and any(pdb in f for pdb in pdbs)]
    # Split into chunks
    chunk_size = chunk_sizes[group]
    for i in range(0, len(fa_files), chunk_size):
        folder_name = f"{group}_{i//chunk_size+1}"
        folder_path = os.path.join(output_base, folder_name)
        os.makedirs(folder_path, exist_ok=True)
        for fa_file in fa_files[i:i+chunk_size]:
            shutil.copy(os.path.join(fa_folder, fa_file), folder_path)

In [14]:
import os

parent_folder = './insectipep'

for subfolder in os.listdir(parent_folder):
    
    subfolder_path = os.path.join(parent_folder, subfolder)
    
    if os.path.isdir(subfolder_path):
        num_files = len([
            f for f in os.listdir(subfolder_path)
            if os.path.isfile(os.path.join(subfolder_path, f))
        ])
        print(f"{subfolder}: {num_files} files")
        number += num_files
print(f"Total files: {number}")

split: 0 files
Total files: 5768


In [11]:
import os
import shutil
import pandas as pd

# Load the dataframe
prot_df = pd.read_csv("/Users/vitran/Documents/Work/Github/Insectipep/data/final_target_list.csv")

# Classify by sequence length
def classify_len(seq):
    l = len(seq)
    if l <= 400:
        return "S"
    elif l <= 700:
        return "M"
    else:
        return "L"

prot_df["Group"] = prot_df["Sequence"].apply(classify_len)

# Set chunk sizes
chunk_sizes = {"S": 60, "M": 30, "L": 15}

# Folder containing .fa files
fa_folder = "/Users/vitran/Documents/Work/Github/ColabFold_data/insectipep"
output_base = "/Users/vitran/Documents/Work/Github/ColabFold_data/insectipep/split"

# Ensure output base exists
os.makedirs(output_base, exist_ok=True)

for group in ["S", "M", "L"]:
    # Get pdbs for this group
    pdbs = prot_df[prot_df["Group"] == group]["PDB"].tolist()
    # Find matching .fa files
    fa_files = [f for f in os.listdir(fa_folder) if f.endswith(".fa") and any(pdb in f for pdb in pdbs)]
    # Split into chunks
    chunk_size = chunk_sizes[group]
    for i in range(0, len(fa_files), chunk_size):
        folder_name = f"{group}_{i//chunk_size+1}"
        folder_path = os.path.join(output_base, folder_name)
        os.makedirs(folder_path, exist_ok=True)
        for fa_file in fa_files[i:i+chunk_size]:
            src = os.path.join(fa_folder, fa_file)
            dst = folder_path
            try:
                shutil.move(src, dst)
            except FileNotFoundError:
                pass

In [ ]:
from pathlib import Path
import zipfile
import shutil
from typing import List, Tuple, Optional, Union

def extract_a3m_from_zips(
    src_dir: Union[str, Path],
    dest_dir: Union[str, Path],
    extract_base: Optional[Union[str, Path]] = None,
    overwrite: bool = False,
) -> List[Tuple[Path, Path, Path]]:
    """
    For each .zip file in src_dir:
      - create an extraction folder (extract_base/<zip-stem>)
      - extract the zip there
      - find all .a3m files (case-insensitive) inside the extracted folder
      - copy each .a3m to dest_dir (auto-rename to avoid collisions unless overwrite=True)

    Args:
        src_dir: directory containing .zip files
        dest_dir: directory where .a3m files will be copied
        extract_base: base directory where zip contents will be extracted.
                      If None, uses src_dir / "extracted"
        overwrite: if True, overwrite files in dest_dir that share the same filename

    Returns:
        A list of tuples (zip_path, original_a3m_path, copied_dest_path)
    """
    src_dir = Path(src_dir)
    dest_dir = Path(dest_dir)
    extract_base = Path(extract_base) if extract_base is not None else src_dir / "extracted"

    if not src_dir.exists() or not src_dir.is_dir():
        raise ValueError(f"src_dir does not exist or is not a directory: {src_dir}")

    dest_dir.mkdir(parents=True, exist_ok=True)
    extract_base.mkdir(parents=True, exist_ok=True)

    copied_records: List[Tuple[Path, Path, Path]] = []

    # iterate zip files (case-insensitive .zip)
    for zip_path in sorted(src_dir.iterdir()):
        if not zip_path.is_file():
            continue
        if zip_path.suffix.lower() != ".zip":
            continue

        out_dir = extract_base / zip_path.stem
        out_dir.mkdir(parents=True, exist_ok=True)

        try:
            with zipfile.ZipFile(zip_path, "r") as zf:
                zf.extractall(out_dir)
        except zipfile.BadZipFile:
            print(f"[WARN] Bad zip file, skipping: {zip_path}")
            continue
        except Exception as e:
            print(f"[WARN] Failed to extract {zip_path}: {e}")
            continue

        # find .a3m files (case-insensitive)
        for candidate in out_dir.rglob("*"):
            if not candidate.is_file():
                continue
            if candidate.suffix.lower() != ".a3m":
                continue

            target = dest_dir / candidate.name

            if target.exists():
                if overwrite:
                    # will overwrite
                    pass
                else:
                    # find a free name: name_1.a3m, name_2.a3m, ...
                    base = candidate.stem
                    ext = candidate.suffix
                    i = 1
                    while True:
                        candidate_name = f"{base}_{i}{ext}"
                        candidate_target = dest_dir / candidate_name
                        if not candidate_target.exists():
                            target = candidate_target
                            break
                        i += 1

            shutil.copy2(candidate, target)
            copied_records.append((zip_path, candidate, target))

    return copied_records

src = "ucloud/"
dst = "ucloud/a3m/"
records = extract_a3m_from_zips(src, dst, extract_base=None, overwrite=False)


Copied 2 files:
